# Episode VII Notebook

The primary purpose of this episode is to connect to Microsoft Azure Storage to show how to upload and download a CSV file.

First, load the .ini file via `configparser`.

In [4]:
import configparser

config = configparser.ConfigParser()
config.read('episode_vii.ini')  

['episode_vii.ini']

## Have a Look at the CSV Data

The data is nothing special, by design, so as to not dilute the Azure Storage topic.

In [6]:
import pandas as pd 
pd.read_csv('./data/regression.csv')

,x0,x1,x2,x3,x4,x5,y
0,-17.933519,6.559220,2.422468,-27.737392,-12.080601,-3.892934,1
1,-37.214754,10.774930,5.404072,21.354738,0.612690,-3.093533,0
2,0.330441,-19.609972,-1.331804,-15.153892,19.710240,19.077300,0
3,-13.709765,-8.011390,-1.536483,23.129497,27.880879,20.573991,0
4,-4.202598,7.076210,8.881550,23.600777,26.232164,-14.462320,1
...,...,...,...,...,...,...,...
4995,-16.890267,-16.018121,3.125934,11.209671,21.754189,-14.794522,1
4996,3.071703,-16.908284,-0.782860,-17.203593,21.367046,0.616527,0
4997,-6.771443,9.255075,-4.606123,-24.855697,-53.425928,-2.635130,0
4998,-4.455558,-21.087259,-0.817243,-11.115997,-13.093764,-6.064185,0


## Connect to the Storage Account and get a Blob Service Client Object

I want to save this CSV to Azure Storage so my fictional FranklyAI team can all see the raw data.

In [5]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

storage_connection_string = config['azure.settings']['storage_connection_string']

blob_service_client = None
blob_service_client = BlobServiceClient.from_connection_string(storage_connection_string)
print('Got the Blob Service Client object.')

Got the Blob Service Client object.


## Create a Blob Container 

This is what holds the CSV file and is effectively the I/O.

This code only needs to be run once - a container can be reused. NOTE: save the UUID, which   
becomes the name of your container. Put this in the .ini file for later use so you don't  
have to create the container each time.

In [9]:
import uuid

# Run the following the first time, to create the container.
# Create a unique name for the container
container_name = str(uuid.uuid4())

# Create the container
container_client = blob_service_client.create_container(container_name)
print('Container created:', container_name)

Container created: 8a290f7b-45bb-4e37-ba37-cbdc3cc3a78c


### Deleting a Blob Container

If you want/need to delete to container, here's some code to do that.

In [7]:
blob_service_client.delete_container('YOUR CONTAINER NAME')

## Get a Container Client from the Blob Service Client and Upload the File

This code assumes the container name from the last code block was saved in the .ini file.

In [10]:
csv_file_name = './data/regression.csv'

if container_name == None:
    container_name = config['azure.settings']['storage_container_name']

container_client = None
for container in blob_service_client.list_containers():
    if container.name == container_name:
        container_client = blob_service_client.get_container_client(container)

csv_file = open(csv_file_name, 'rb')
container_client.upload_blob(name='regression.csv', data=csv_file)
print('Blob upload complete.')

Blob upload complete.


## Get the CSV and Create DataFrame Using Only Data
There's no need to save a .csv file - the stream of data (which can be bytes or a String) is  
sufficient. No duplicate CSV files!

In [11]:
import pandas as pd 
from azure.storage.blob import BlobProperties
from io import StringIO

download_stream = container_client.download_blob(blob=BlobProperties(name='regression.csv'))
download_str = download_stream.content_as_text()
df = pd.read_csv(StringIO(download_str))
print('Loaded the CSV data into a DataFrame.')

Loaded the CSV data into a DataFrame.


### Download the CSV as a File

If you want/need to actually download and save the CSV file, you can do that with the following code.

In [ ]:
from azure.storage.blob import BlobProperties
download_file = the_cwd + './data/regression_download.csv'
with open(download_file, "wb") as my_blob:
    download_stream = container_client.download_blob(BlobProperties(name='regression.csv'))
    my_blob.write(download_stream.readall())

## Look at the Downloaded CSV

Make sure it's OK and the same as the original.

In [12]:
print(df)

             x0         x1        x2         x3         x4         x5  y
0    -17.933519   6.559220  2.422468 -27.737392 -12.080601  -3.892934  1
1    -37.214754  10.774930  5.404072  21.354738   0.612690  -3.093533  0
2      0.330441 -19.609972 -1.331804 -15.153892  19.710240  19.077300  0
3    -13.709765  -8.011390 -1.536483  23.129497  27.880879  20.573991  0
4     -4.202598   7.076210  8.881550  23.600777  26.232164 -14.462320  1
...         ...        ...       ...        ...        ...        ... ..
4995 -16.890267 -16.018121  3.125934  11.209671  21.754189 -14.794522  1
4996   3.071703 -16.908284 -0.782860 -17.203593  21.367046   0.616527  0
4997  -6.771443   9.255075 -4.606123 -24.855697 -53.425928  -2.635130  0
4998  -4.455558 -21.087259 -0.817243 -11.115997 -13.093764  -6.064185  0
4999 -10.751591  -6.059221 -0.173580 -16.036675 -19.536580   3.649812  0

[5000 rows x 7 columns]


# References:
* https://franklyai.medium.com/fix-modulenotfounderror-no-module-named-ruamel-when-importing-azureml-core-7264d1860612
* https://docs.microsoft.com/en-us/azure/machine-learning/how-to-access-data
* https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py
* https://code.visualstudio.com/docs/python/environments
* https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py
* https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-python